# Simple baseline model for beginners using sklearn pipeline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
train_data.head()

In [ ]:
y=train_data.target
X=train_data.drop(['id','target'],axis=1)
X.head()

Get a list of numerical and categorical columns

In [ ]:
numerical_cols=[cname for cname in X.columns if X[cname].dtype in ['int64','float64','int']]
numerical_cols

In [ ]:
categorical_cols=[cname for cname in X.columns if X[cname].dtype =='object']
categorical_cols

Create sklearn pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

numerical_transformer = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('impute',SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps = [
    ('imputer',SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor=ColumnTransformer(transformers=[
    ('num',numerical_transformer,numerical_cols),
    
    ('cat',categorical_transformer,categorical_cols),
     
])
model=XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=0)
my_regressor=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])

Train the model

In [ ]:
my_regressor.fit(X,y)

Import the test test and create submission file.

In [ ]:
X_test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
uid=X_test.id
X_test.drop(['id'],axis=1,inplace=True)
X_test.head()

In [ ]:
preds_test=my_regressor.predict(X_test)
preds_test

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'id': uid,
                       'target': preds_test})
output.to_csv('submission_1.csv', index=False)

In [ ]:
output.head()